In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
# import sumo as sm
import os
from traffic_initialization import *
import sys
from bs4 import BeautifulSoup
import tqdm
import pandas as pd

# New handy interface is here!
# Check interface.py for possible parameters
from interface import Simulation
%env SUMO_HOME=/usr/share/sumo
from datetime import datetime
%load_ext autotime

env: SUMO_HOME=/usr/share/sumo
time: 53.6 µs (started: 2024-04-24 19:22:42 +03:00)


# Optimization with annealing

In [2]:
# simple throughput function with two gaps parameters

def loss(d0, d1, N):

    ans = 0
    
    for n in range(10):

        sim = Simulation(start_road='E0', end_road='E2', N=N, work_dir='./xml_data/test6', net_data='test6.net.xml')
        sim.set_calibrator(road='E0', gap=3)
        sim.set_calibrator(road='E1', lane=['0', '1'], gap=d0)
        sim.set_calibrator(road='E1', lane=['2'], gap=d1)
        sim.set_calibrator(road='E1', lane=['3'], gap=2)
        sim.set_calibrator(road='E2', gap=1.5)
        sim.run()
        ans += sim.get_mean_throughput()

    return -ans/10

time: 1.46 ms (started: 2024-04-24 19:20:31 +03:00)


# Optimization with ML

In [2]:
from Some_models import OptimalLoss_Q, OptimalDist

time: 182 ms (started: 2024-04-24 19:22:46 +03:00)


In [3]:
# complex throughput function with three gaps parameters

def loss(d0, d1, d2, N):

    ans = 0
    
    for n in range(10):

        sim = Simulation(start_road='E0', end_road='E2', N=N, work_dir='./xml_data/test6', net_data='test6.net.xml')
        sim.set_calibrator(road='E0', gap=3)
        sim.set_calibrator(road='E1', lane=['0', '1'], gap=d0)
        sim.set_calibrator(road='E1', lane=['2'], gap=d1)
        sim.set_calibrator(road='E1', lane=['3'], gap=2)
        sim.set_calibrator(road='E2', gap=1.5)
        sim.run()
        ans += sim.get_mean_throughput()

    return -ans/10

time: 275 µs (started: 2024-04-24 19:22:48 +03:00)


In [4]:
model = OptimalDist(grid_params=[(10, 101, 25), (10, 101, 25), (10, 101, 25), (10, 101, 25)])
model.fit_func(loss)

time: 1min 32s (started: 2024-04-24 19:22:49 +03:00)


# Creating of csv

In [3]:
throughput(20, 4, 6000)

1.2891872320004407

time: 27.8 s (started: 2024-04-24 18:31:17 +03:00)


In [4]:
throughput(20, 4, 6000)

1.2932371519835026

time: 27.7 s (started: 2024-04-24 18:31:45 +03:00)


In [18]:
cost(30, 20, 10, 6000)

1.2176827071439196

In [19]:
cost(30, 20, 10, 6000)

1.2098426779299611

In [10]:
cost(30, 20, 10, 3)

0.028846153846153848

In [ ]:
while(True):
    # test6
    filename='csv_data/test6.csv'
    csv_data = pd.read_csv(filename)
    
    N = np.random.randint(3000, 8000)
    for i in range(500):
        rand = np.random.rand(3)
        d0 = 10 + 40*rand[0]
        d1 = 10 + 30*rand[1]
        d2 = 3 + 30*rand[2]
        
        csv_data.loc[len(csv_data)] = (d0, d1, d2, N, sim.get_mean_throughput())
        csv_data.to_csv(filename, index=False) 